# NDVI creation
This script loads landsat images (tifs) from a local source (they need to already be downloaded to your own device) and creates NDVI images from them.

Additionally, you will need to download an ownership and fire area map from the github page. Both of these files are located in the **DouglasFireTiffs** folder.
- ***DF_area.tif***
- ***DF_ownership.tif***


In [1]:
# Import packages
from osgeo import gdal
import os
import glob

import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid1 import make_axes_locatabl
import numpy as np
import rasterio
import xarray
import rioxarray
import datetime

## Define paths to data
Below you need to define local paths to your folders where landsat images and ownership and area masks are located.

We suggest you store the landsat images not in your local github clone folder.

In [2]:
# Define filepaths
filepath7 = 'E:/GitHub/GEOG_L7/'
filepath8 = 'E:/GitHub/GEOG_L8/'

filepath_DF_mask = 'E:/GitHub/PostFire_NDVI/DouglasFireTiffs/DF_area.tif'
filepath_DF_ownership = 'E:/GitHub/PostFire_NDVI/DouglasFireTiffs/DF_ownership.tif'

# create rasters from DF_area.tif and DF_ownership
df_mask = rioxarray.open_rasterio(filepath_DF_mask)
df_ownership = rioxarray.open_rasterio(filepath_DF_ownership)

# Define list of Landsat bands
files7 = sorted(glob.glob(filepath7 + '*.tif'))
files8 = sorted(glob.glob(filepath8 + '*.tif'))

## Define functions to process and create NDVI for landsat 7 and landsat 8
- landat7 uses bands 4 and 3 to calculate NDVI
- landsat8 uses bands 4 and 5 to calculate NDVI
- set folder path in EACH function to where you will store ndvi images (make this the same folder). Again we suggest this is not in your local github cloned folder.

In [3]:
def process_ndvi_7(file):
    #strip name from filepath for later
    name = file  # may have to use the following depending on your computer  - file.split("/")[-1]
    savename = file[-24:]  # if you uncommment out above line can comment this out
    xds = rioxarray.open_rasterio(file)
    
    #select band 4 and 3 from landsat 7 tif
    band4 = xds.sel(band=1)
    band3 = xds.sel(band=2)
    #band2 = xds.sel(band=3)
    
    # Compute NDVI
    ndvi = np.divide((band4 - band3), (band4 + band3))
    
    # clip to shape of Douglas Fire
    clipped_ndvi = ndvi.where(df_mask == 1)
    clipped_ndvi = clipped_ndvi.transpose('band', 'y', 'x')
    
    clipped_ndvi.rio.to_raster("E:/GitHub/landsat_ndvi/ndvi_"+savename)
    
def process_ndvi_8(file):
    name = file  # may have to use the following depending on your computer  - file.split("/")[-1]
    savename = file[-24:]  # if you uncommment out above line can comment this out
    xds = rioxarray.open_rasterio(file)
    
    band5 = xds.sel(band=1)
    band4 = xds.sel(band=2)
    #band3 = xds.sel(band=3)
    
    # Compute NDVI
    #p.seterr(divide='ignore', invalid='ignore')
    ndvi = np.divide((band5 - band4), (band5 + band4))
                   
    # clip to shape of Douglas Fire
    clipped_ndvi = ndvi.where(df_mask == 1)
    clipped_ndvi = clipped_ndvi.transpose('band', 'y', 'x')               
    
    clipped_ndvi.rio.to_raster("E:/GitHub/landsat_ndvi/ndvi_"+savename)

### This next line will take long to run!!!
- here you will actually process all the tif images in your landsat folders

In [4]:
for file in files7:
    process_ndvi_7(file)

for file in files8:
    process_ndvi_8(file)